# Code for generating a hexagonal tessellation of a city
Given bookings collection with the lat_lon of sharing cars bookings, the notebook generate a hexagonal grid that cover the spanned area. The hexagons are inserted in the "hexs_gen" collections. 

See the rest of the notebook for parameters and settings

In [1]:
import pandas as pd
import sys
import math
import imp
import pymongo as pym
import json
import numpy as np
import random
import sys
sys.path.insert(0, './lib_python/')

In [2]:
urlMongoDbLocal = "mongodb://localhost:27017";
clientLocal = pym.MongoClient(urlMongoDbLocal)

In [9]:
carSharing = clientLocal["bigDive_test"]
bookingsDB = carSharing["bookings"]
hexs_gen = carSharing["hexs_gen"]

In [10]:
city = "Milano" 

In [11]:
step = 0.4 # step of the exagonal grid

## Finding the box area that cover all the booking area

In [12]:
def find_extreme_latlon(coll, city, field, order = -1, latLon = "lat"):
    if latLon == "lat":
        latLon = 1
    elif latLon == "lon":
        latLon = 0
    else:
        print("error Latlon not equal at 'lan' or 'lon'")
        return 0
    insort = field + ".coordinates." + str(latLon)
    #print(insort, latLon, order)
    return coll.find_one({"city":city}, sort=([(insort, order)]))[field]["coordinates"][latLon]

In [13]:
import shapely
import libHex
import imp 
import folium
name_start_point = "origin_point"
name_end_point = "dest_point"
max_lon = find_extreme_latlon(bookingsDB, city, name_start_point, order = -1,latLon = "lon")
max_lon = max(max_lon, find_extreme_latlon(bookingsDB, city, name_end_point, -1, "lon"))
min_lon = find_extreme_latlon(bookingsDB, city, name_start_point, 1,"lon")
min_lon = min(max_lon, find_extreme_latlon(bookingsDB, city, name_end_point, 1, "lon"))

max_lat = find_extreme_latlon(bookingsDB, city, name_start_point, -1,"lat")
max_lat = max(max_lat, find_extreme_latlon(bookingsDB, city, name_end_point, -1, "lat"))
min_lat = find_extreme_latlon(bookingsDB, city, name_start_point, 1,"lat")
min_lat = min(max_lat, find_extreme_latlon(bookingsDB, city, name_end_point, 1, "lat"))

listpoint =[[max_lat, min_lon], [max_lat,max_lon],[min_lat,max_lon],[min_lat,min_lon], [max_lat,min_lon]]
listpoint = [ [float(x[0]), float(x[1])] for x in listpoint]

map_folium =  folium.Map(location = [ (max_lat + min_lat)/2, (max_lon + min_lon)/2], zoom_start=11)
line = folium.PolyLine(listpoint)
map_folium.add_child(line)
map_folium

## Generating the hexagonal grid

In [14]:
cell = 0.4
(grid_multipoligon, list_hex) = libHex.hexagonalGrid((min_lon,max_lat,max_lon,min_lat), cell = cell, city = city)

### Removing the hexagons with no bookings inside

In [16]:
list_hex_with_cars = []
count = 0
bookingsDB.c
for h in list_hex:
    find_start = {"city" : city}
    find_start[name_start_point] = {
            "$geoWithin":{
                "$geometry": h["hex"]
            }}
    find_end = {"city" : city}
    find_end[name_end_point] = {
            "$geoWithin":{
                "$geometry": h["hex"]
            }}

    c1 = bookingsDB.find(find_start).count()
    c2 = bookingsDB.find(find_end).count()
    if c1 + c2 > 0:
        list_hex_with_cars.append(h)
    count += 1
    if count % 10 ==0:
        print("{0}".format(count), end="\r")

In [17]:
print("number of hexs with bookings {0}".format(len(list_hex_with_cars)))
map_folium =  folium.Map(location = [ (max_lat + min_lat)/2, (max_lon + min_lon)/2], zoom_start=11)
for h in list_hex_with_cars:
    hex_ = folium.GeoJson(h["hex"])
    map_folium.add_child(hex_)
map_folium

number of hexs with bookings 1156


## inserting in the database

In [18]:
hexs_gen.insert_many(list_hex_with_cars)

In [19]:
hexs_gen.find_one()

{'_id': ObjectId('5b33fa8827691d1442c46caa'),
 'city': 'Milano',
 'hex': {'coordinates': [[[9.084975138449533, 45.477074451794614],
    [9.083694887037502, 45.47551601836925],
    [9.081134384213438, 45.47551601836925],
    [9.079854132801408, 45.477074451794614],
    [9.081134384213438, 45.47863288521998],
    [9.083694887037502, 45.47863288521998],
    [9.084975138449533, 45.477074451794614]]],
  'type': 'Polygon'},
 'point': {'coordinates': [9.08241463562547, 45.477074451794614],
  'type': 'Point'},
 'pos': 18}

In [20]:
hexs_gen.create_index([("point", pym.GEOSPHERE)])
hexs_gen.create_index([("hex", pym.GEOSPHERE)])
hexs_gen.create_index([("city", 1)])
hexs_gen.create_index([("pos", 1)])

'pos_1'